In [1]:
import torch
torch.cuda.device_count()

1

In [ ]:
#sometimes google give me a gpu runtime, don't know why

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#!!!I changed the base folder (no 4471)

In [3]:
BASEFOLDER = '/content/drive/MyDrive/Colab Notebooks/proj-NAFNet/multi-output' 
import os
os.chdir(BASEFOLDER)

In [4]:
def base(rel_path):
  return f"{BASEFOLDER}/{rel_path}"

In [5]:
os.chdir(base("NAFNet-main"))

In [ ]:
!pip install -r requirements.txt
!python setup.py develop --no_cuda_ext
!pip install einops

In [7]:
os.chdir(base("Restormer-main"))

In [8]:
!python setup.py develop --no_cuda_ext

running develop
running egg_info
writing basicsr_restormer.egg-info/PKG-INFO
writing dependency_links to basicsr_restormer.egg-info/dependency_links.txt
writing top-level names to basicsr_restormer.egg-info/top_level.txt
adding license file 'LICENSE.md'
writing manifest file 'basicsr_restormer.egg-info/SOURCES.txt'
/usr/local/lib/python3.8/dist-packages/torch/utils/cpp_extension.py:411: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
running build_ext
Creating /usr/local/lib/python3.8/dist-packages/basicsr-restormer.egg-link (link to .)
Adding basicsr-restormer 1.2.0+10018c6 to easy-install.pth file

Installed /content/drive/MyDrive/Colab Notebooks/proj-NAFNet/multi-output/Restormer-main
Processing dependencies for basicsr-restormer==1.2.0+10018c6
Finished processing dependencies for basicsr-restormer==1.2.0+10018c6


In [9]:
!git config --add core.bigFileThreshold 1
!git clone --branch upload https://github.com/TonyNLM/pics /content/pics

Cloning into '/content/pics'...
remote: Enumerating objects: 52959, done.
remote: Counting objects: 100% (7534/7534), done.
remote: Compressing objects: 100% (7374/7374), done.
remote: Total 52959 (delta 177), reused 7517 (delta 160), pack-reused 45425
Receiving objects: 100% (52959/52959), 3.33 GiB | 16.50 MiB/s, done.
Resolving deltas: 100% (1456/1456), done.
Checking out files: 100% (39769/39769), done.


In [31]:
!cp -r /content/pics/val/source /content/pics/val/target
!cp -r /content/pics/val/blur /content/pics/val/input

Test images too big, running out of memory if we don't crop them.

In [32]:
from PIL import Image

def crop(fname):
  i = Image.open(fname)
  i.crop((0,0,360,360)).save(fname)

def cropdir(inputdir):
  for fname in os.listdir(inputdir):
    crop(f"{inputdir}/{fname}")

cropdir("/content/pics/val/target")
cropdir("/content/pics/val/input")


In [14]:
!cp -r /content/pics/test/source /content/pics/test/target
!cp -r /content/pics/test/blur /content/pics/test/input

In [15]:
cropdir("/content/pics/test/target")
cropdir("/content/pics/test/input")

In [39]:
#before training
os.chdir(base("Restormer-main/Motion_Deblurring"))
!python test.py --input_dir /content/pics/val/input --result_dir /content/pics/test/target

===>Testing using weights:  ./pretrained_models/motion_deblurring.pth
100% 1000/1000 [09:30<00:00,  1.75it/s]
PSNR: 32.59869593089285, SSIM: 0.9646566426921636


In [22]:
#before training
os.chdir(base("Restormer-main/Motion_Deblurring"))
!python test.py --input_dir /content/pics/test/input --result_dir /content/pics/test/target

===>Testing using weights:  ./pretrained_models/motion_deblurring.pth
100% 1000/1000 [09:34<00:00,  1.74it/s]
PSNR: 32.76182567129378, SSIM: 0.9650098627302796


In [23]:
os.chdir(base("Restormer-main"))

In [ ]:
#yml checking
!cat Motion_Deblurring/Options/BaseRestormer_mini.yml

In [38]:
!python -m torch.distributed.launch --nproc_per_node=1 basicsr_restormer/train.py -opt Motion_Deblurring/Options/BaseRestormer_mini.yml --launcher pytorch

/usr/local/lib/python3.8/dist-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
init dist ..  pytorch
Path already exists. Rename it to /content/drive/MyDrive/Colab Notebooks/proj-NAFNet/multi-output/Restormer-main/experiments/Mini_Restormer_archived_20221203_063125
Path already exists. Rename it to tb_logger/Mini_Restormer_archived_20221203_063125
2022-12-03 06:31:25,417 INFO: 
                ____                _       _____  ____
               / __ ) ____ _ _____ (_)_____/ ___/ / __ \
              / __  |/ __ `// ___// // ___/\__ \ / /_/ /
             / /_/ // /_/ /(__  )/ // /__ ___/ // _, _/
  

In [42]:
os.chdir(base("Restormer-main/Motion_Deblurring"))
!python test.py --input_dir /content/pics/test/input --result_dir /content/pics/test/target --weights ../experiments/Mini_Restormer/models/net_g_6000.pth

===>Testing using weights:  ../experiments/Mini_Restormer/models/net_g_6000.pth
100% 1000/1000 [09:31<00:00,  1.75it/s]
PSNR: 27.05777612979615, SSIM: 0.918818864285946
